In [1]:
import regex as re

# 特殊标记
special_tokens = ["<|endoftext|>"]

# 读取数据
with open("./data/TinyStoriesV2-GPT4-valid.txt", 'r', encoding='utf-8') as f:
    data = f.read()

# 处理特殊标记
delimiter_pattern = "|".join(re.escape(s) for s in special_tokens)
parts = re.split(delimiter_pattern, data)
parts = parts[:3]
parts

['u don\'t have to be scared of the loud dog, I\'ll protect you". The mole felt so safe with the little girl. She was very kind and the mole soon came to trust her. He leaned against her and she kept him safe. The mole had found his best friend.\n',
 '\nOnce upon a time, in a warm and sunny place, there was a big pit. A little boy named Tom liked to play near the pit. One day, Tom lost his red ball. He was very sad.\nTom asked his friend, Sam, to help him search for the ball. They looked high and low, but they could not find the ball. Tom said, "I think my ball fell into the pit."\nSam and Tom went close to the pit. They were scared, but they wanted to find the red ball. They looked into the pit, but it was too dark to see. Tom said, "We must go in and search for my ball."\nThey went into the pit to search. It was dark and scary. They could not find the ball. They tried to get out, but the pit was too deep. Tom and Sam were stuck in the pit. They called for help, but no one could hear 

In [2]:

# 正则表达式用于分割文本
PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
tokens = []
token_number = {}

# 分词并统计频率
for part in parts:
    tokens += re.findall(PAT, part)

# 统计每个 token 的出现次数
for token in tokens:
    if token in token_number:
        token_number[token] += 1
    else: 
        token_number[token] = 1
token_number

{'u': 1,
 ' don': 1,
 "'t": 1,
 ' have': 3,
 ' to': 21,
 ' be': 1,
 ' scared': 5,
 ' of': 6,
 ' the': 28,
 ' loud': 2,
 ' dog': 1,
 ',': 26,
 ' I': 5,
 "'ll": 1,
 ' protect': 1,
 ' you': 5,
 '".': 1,
 ' The': 2,
 ' mole': 3,
 ' felt': 4,
 ' so': 2,
 ' safe': 3,
 ' with': 7,
 ' little': 2,
 ' girl': 1,
 '.': 69,
 ' She': 8,
 ' was': 13,
 ' very': 3,
 ' kind': 1,
 ' and': 31,
 ' soon': 1,
 ' came': 2,
 ' trust': 1,
 ' her': 4,
 ' He': 8,
 ' leaned': 1,
 ' against': 1,
 ' she': 2,
 ' kept': 1,
 ' him': 7,
 ' had': 2,
 ' found': 1,
 ' his': 7,
 ' best': 1,
 ' friend': 2,
 '\n': 21,
 'Once': 1,
 ' upon': 1,
 ' a': 11,
 ' time': 1,
 ' in': 5,
 ' warm': 1,
 ' sunny': 1,
 ' place': 1,
 ' there': 1,
 ' big': 2,
 ' pit': 9,
 ' A': 1,
 ' boy': 1,
 ' named': 1,
 ' Tom': 11,
 ' liked': 3,
 ' play': 1,
 ' near': 1,
 ' One': 1,
 ' day': 1,
 ' lost': 1,
 ' red': 2,
 ' ball': 7,
 ' sad': 3,
 'Tom': 4,
 ' asked': 3,
 ' Sam': 2,
 ' help': 2,
 ' search': 3,
 ' for': 6,
 ' They': 16,
 ' looked': 3,
 ' high

In [ ]:
# 构建词汇表
vocab = {}
current_id = 0
# 将特殊 Token 字符串编码为 bytes，并加入词汇表
    # 每个字节都作为长度为 1 的 bytes 对象
for i in range(256):
    vocab[current_id] = bytes([i])  # 将整数 i 转换为单字节的 bytes 对象
    current_id += 1
    # 将特殊 Token 字符串编码为 bytes，并加入词汇表
for token_str in special_tokens:
    vocab[current_id] = token_str.encode('utf-8')  # 使用 .encode 将字符串转换为字节
    current_id += 1

vocab

 








	























 
!
"
#
$
%
&
'
(
)
*
+
,
-
.
/
0
1
2
3
4
5
6
7
8
9
:
;
<
=
>
?
@
A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
U
V
W
X
Y
Z
[
\
]
^
_
`
a
b
c
d
e
f
g
h
i
j
k
l
m
n
o
p
q
r
s
t
u
v
w
x
y
z
{
|
}
~

































 
¡
¢
£
¤
¥
¦
§
¨
©
ª
«
¬
­
®
¯
°
±
²
³
´
µ
¶
·
¸
¹
º
»
¼
½
¾
¿
À
Á
Â
Ã
Ä
Å
Æ
Ç
È
É
Ê
Ë
Ì
Í
Î
Ï
Ð
Ñ
Ò
Ó
Ô
Õ
Ö
×
Ø
Ù
Ú
Û
Ü
Ý
Þ
ß
à
á
â
ã
ä
å
æ
ç
è
é
ê
ë
ì
í
î
ï
ð
ñ
ò
ó
ô
õ
ö
÷
ø
ù
ú
û
ü
ý
þ
ÿ


{0: '<|endoftext|>',
 1: '\x00',
 2: '\x01',
 3: '\x02',
 4: '\x03',
 5: '\x04',
 6: '\x05',
 7: '\x06',
 8: '\x07',
 9: '\x08',
 10: '\t',
 11: '\n',
 12: '\x0b',
 13: '\x0c',
 14: '\r',
 15: '\x0e',
 16: '\x0f',
 17: '\x10',
 18: '\x11',
 19: '\x12',
 20: '\x13',
 21: '\x14',
 22: '\x15',
 23: '\x16',
 24: '\x17',
 25: '\x18',
 26: '\x19',
 27: '\x1a',
 28: '\x1b',
 29: '\x1c',
 30: '\x1d',
 31: '\x1e',
 32: '\x1f',
 33: ' ',
 34: '!',
 35: '"',
 36: '#',
 37: '$',
 38: '%',
 39: '&',
 40: "'",
 41: '(',
 42: ')',
 43: '*',
 44: '+',
 45: ',',
 46: '-',
 47: '.',
 48: '/',
 49: '0',
 50: '1',
 51: '2',
 52: '3',
 53: '4',
 54: '5',
 55: '6',
 56: '7',
 57: '8',
 58: '9',
 59: ':',
 60: ';',
 61: '<',
 62: '=',
 63: '>',
 64: '?',
 65: '@',
 66: 'A',
 67: 'B',
 68: 'C',
 69: 'D',
 70: 'E',
 71: 'F',
 72: 'G',
 73: 'H',
 74: 'I',
 75: 'J',
 76: 'K',
 77: 'L',
 78: 'M',
 79: 'N',
 80: 'O',
 81: 'P',
 82: 'Q',
 83: 'R',
 84: 'S',
 85: 'T',
 86: 'U',
 87: 'V',
 88: 'W',
 89: 'X',
 90: 'Y'

In [9]:

# 初始化合并循环
freq_dict = {}  # 使用更好的命名
merges = []
new_token_number = {}

# 将 string 转换为 tuple[bytes]
for token, count in token_number.items():
    token_bytes_tuple = tuple(c for c in token)  # 将字符串 token 转换为字节串并转为 tuple
    new_token_number[token_bytes_tuple] = count
new_token_number

{('u',): 1,
 (' ', 'd', 'o', 'n'): 1,
 ("'", 't'): 1,
 (' ', 'h', 'a', 'v', 'e'): 3,
 (' ', 't', 'o'): 21,
 (' ', 'b', 'e'): 1,
 (' ', 's', 'c', 'a', 'r', 'e', 'd'): 5,
 (' ', 'o', 'f'): 6,
 (' ', 't', 'h', 'e'): 28,
 (' ', 'l', 'o', 'u', 'd'): 2,
 (' ', 'd', 'o', 'g'): 1,
 (',',): 26,
 (' ', 'I'): 5,
 ("'", 'l', 'l'): 1,
 (' ', 'p', 'r', 'o', 't', 'e', 'c', 't'): 1,
 (' ', 'y', 'o', 'u'): 5,
 ('"', '.'): 1,
 (' ', 'T', 'h', 'e'): 2,
 (' ', 'm', 'o', 'l', 'e'): 3,
 (' ', 'f', 'e', 'l', 't'): 4,
 (' ', 's', 'o'): 2,
 (' ', 's', 'a', 'f', 'e'): 3,
 (' ', 'w', 'i', 't', 'h'): 7,
 (' ', 'l', 'i', 't', 't', 'l', 'e'): 2,
 (' ', 'g', 'i', 'r', 'l'): 1,
 ('.',): 69,
 (' ', 'S', 'h', 'e'): 8,
 (' ', 'w', 'a', 's'): 13,
 (' ', 'v', 'e', 'r', 'y'): 3,
 (' ', 'k', 'i', 'n', 'd'): 1,
 (' ', 'a', 'n', 'd'): 31,
 (' ', 's', 'o', 'o', 'n'): 1,
 (' ', 'c', 'a', 'm', 'e'): 2,
 (' ', 't', 'r', 'u', 's', 't'): 1,
 (' ', 'h', 'e', 'r'): 4,
 (' ', 'H', 'e'): 8,
 (' ', 'l', 'e', 'a', 'n', 'e', 'd'): 1,
 (' 

In [ ]:

# 迭代合并频率最高的字符对
while current_id <= 270:
    freq_dict = {}  # 每次循环都需要重新统计频率

    # 统计 token 中字符对的频率
    for token in new_token_number:
        number = len(token)
        for i in range(number - 1):
            pair = (token[i], token[i + 1])  # 创建一个元组
            if pair in freq_dict:
                freq_dict[pair] += new_token_number[token]  # 累加频率
            else:
                freq_dict[pair] = new_token_number[token]  # 初始化为当前的值

    # 找出频率最高的字符对 (pair, frequency)
    max_pair = max(freq_dict.items(), key=lambda x: x[1])

    # 将频率最高的字符对添加到 merges 列表中
    merges.append(max_pair[0])

    # 更新 vocab 字典，将合并后的字符对替换为新的 token
    merged_token = max_pair[0][0] + max_pair[0][1]  # 合并后的字符对变成一个新的 token
    vocab[current_id] = merged_token
    current_id += 1

    # 更新每个 token，将频率最高的字符对替换为合并后的新 token
    new_tokens = {}
    for token in new_token_number:
        updated_token = []
        i = 0
        while i < len(token) - 1:
            pair = (token[i], token[i + 1])
            if pair == max_pair[0]:  # 如果当前字符对是频率最高的字符对
                updated_token.append(merged_token)  # 替换为新 token
                i += 2  # 跳过已经合并的字符对
            else:
                updated_token.append(token[i])  # 保留原字符
                i += 1

        # 如果 token 的最后一个字符没有被处理过，加入更新后的 token 中
        if i < len(token):
            updated_token.append(token[i])

        # 生成新的 token
        new_tokens[tuple(updated_token)] = new_token_number[token]  # 更新后的 token

    # 更新 new_token_number 为新的 token 字典
    new_token_number = new_tokens

# 输出合并结果
print("最终词汇表:", vocab)
print("合并列表:", merges)


最终词汇表: {0: '<|endoftext|>', 1: '\x00', 2: '\x01', 3: '\x02', 4: '\x03', 5: '\x04', 6: '\x05', 7: '\x06', 8: '\x07', 9: '\x08', 10: '\t', 11: '\n', 12: '\x0b', 13: '\x0c', 14: '\r', 15: '\x0e', 16: '\x0f', 17: '\x10', 18: '\x11', 19: '\x12', 20: '\x13', 21: '\x14', 22: '\x15', 23: '\x16', 24: '\x17', 25: '\x18', 26: '\x19', 27: '\x1a', 28: '\x1b', 29: '\x1c', 30: '\x1d', 31: '\x1e', 32: '\x1f', 33: ' ', 34: '!', 35: '"', 36: '#', 37: '$', 38: '%', 39: '&', 40: "'", 41: '(', 42: ')', 43: '*', 44: '+', 45: ',', 46: '-', 47: '.', 48: '/', 49: '0', 50: '1', 51: '2', 52: '3', 53: '4', 54: '5', 55: '6', 56: '7', 57: '8', 58: '9', 59: ':', 60: ';', 61: '<', 62: '=', 63: '>', 64: '?', 65: '@', 66: 'A', 67: 'B', 68: 'C', 69: 'D', 70: 'E', 71: 'F', 72: 'G', 73: 'H', 74: 'I', 75: 'J', 76: 'K', 77: 'L', 78: 'M', 79: 'N', 80: 'O', 81: 'P', 82: 'Q', 83: 'R', 84: 'S', 85: 'T', 86: 'U', 87: 'V', 88: 'W', 89: 'X', 90: 'Y', 91: 'Z', 92: '[', 93: '\\', 94: ']', 95: '^', 96: '_', 97: '`', 98: 'a', 99: 'b',

In [14]:
# 原始字符元组列表
merges = [(' ', 't'), ('h', 'e'), (' ', 'a'), (' ', 's'), (' t', 'he'),
         (' ', 'w'), ('e', 'd'), ('n', 'd'), (' t', 'o'), (' ', 'b'),
         (' a', 'nd'), (' ', 'T'), (' ', 'h'), ('a', 'r')]

# 将字符元组转换为字节元组，逐字符处理
byte_merges = []
for c1, c2 in merges:
    byte_c1 = bytes([ord(c) for c in c1])  # 将每个字符转换为字节
    byte_c2 = bytes([ord(c) for c in c2])  # 将每个字符转换为字节
    byte_merges.append((byte_c1, byte_c2))

# 输出转换后的字节元组列表
print(byte_merges)

[(b' ', b't'), (b'h', b'e'), (b' ', b'a'), (b' ', b's'), (b' t', b'he'), (b' ', b'w'), (b'e', b'd'), (b'n', b'd'), (b' t', b'o'), (b' ', b'b'), (b' a', b'nd'), (b' ', b'T'), (b' ', b'h'), (b'a', b'r')]


In [ ]:
# 频率字典
freq_dict = {
    'lo': 7, 'ow': 7, 'we': 8, 'er': 2, 'wi': 3, 'id': 3, 
    'de': 3, 'es': 9, 'st': 9, 'ne': 6, 'ew': 6
}

# 找出频率最高的字符对，若频率相同，则按字母顺序选择较大的字节对
max_pair = max(freq_dict.items(), key=lambda x: (x[1], x[0][0], x[0][1]))
print(max_pair)


('es', 9)


In [26]:
bytes([1])

b'\x01'